In [23]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint as pprint 
import os
import pymongo
from copy import deepcopy

In [24]:
#Beginning Scrapping

In [25]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [26]:
base_url = 'https://www.payscale.com/cost-of-living-calculator/'
browser.visit(base_url)

In [27]:
base_html = browser.html
base_soup = bs(base_url, 'html.parser')

In [28]:
print(base_soup.prettify())

https://www.payscale.com/cost-of-living-calculator/



In [29]:
city_list = ['Washington-Seattle', 'Oregon-Portland', 'Arizona-Phoenix', 'Colorado-Denver']
table_dfs =[]
for city in city_list:
    df_name = city
    table_dfs.append(df_name)
    
table_dfs 

['Washington-Seattle', 'Oregon-Portland', 'Arizona-Phoenix', 'Colorado-Denver']

In [30]:
count = 0
pages = []
for page in range(2, 6):
    page_url = f"{base_url}{city_list[count]}"
    count = count + 1
    pages.append(page_url)
    print(page_url)    

https://www.payscale.com/cost-of-living-calculator/Washington-Seattle
https://www.payscale.com/cost-of-living-calculator/Oregon-Portland
https://www.payscale.com/cost-of-living-calculator/Arizona-Phoenix
https://www.payscale.com/cost-of-living-calculator/Colorado-Denver


In [31]:
print(pages)

['https://www.payscale.com/cost-of-living-calculator/Washington-Seattle', 'https://www.payscale.com/cost-of-living-calculator/Oregon-Portland', 'https://www.payscale.com/cost-of-living-calculator/Arizona-Phoenix', 'https://www.payscale.com/cost-of-living-calculator/Colorado-Denver']


In [32]:
headers = ['Median Home Price','Median Rent','Energy Bill','Phone Bill','Gas','Loaf of Bread', \
           'Gallon of Milk','Carton of Eggs','Bunch of Bananas','Hamburger',"Doctor's Visit",\
           'Dentist Visit','Optometrist Visit','Rx Drug','Veterinary Visit']
count = 0
lifestyle_df = pd.DataFrame()
for page in pages:
   browser.visit(page)
   western_region = browser.html
   western_soup = bs(western_region, 'html.parser')
   values = []
   for info in western_soup.find_all('div', class_='itemized__item-container'):
       for v in info.find_all('div', class_='itemized__item-value'):
           value = v.text
           values.append(deepcopy(value))
   style_df = pd.DataFrame([{headers[x]:values[x] for x in range(len(headers))}])
   lifestyle_df = lifestyle_df.append(deepcopy(style_df))
df = pd.DataFrame(lifestyle_df)
df['STATES']= city_list
western_data = df.set_index('STATES')
western_data

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
STATES,,,,,,,,,,,,,,,
Washington-Seattle,"$673,874","$2,111 / month",$206.39 / month,$220.00 / month,$3.47 / gallon,$4.22,$2.46,$2.29,$4.26,$5.37,$137.50,$118.13,$128.85,$547.72,$69.64
Oregon-Portland,"$620,783","$1,945 / month",$130.77 / month,$139.40 / month,$2.78 / gallon,$3.84,$2.23,$2.09,$3.87,$4.88,$118.57,$101.86,$111.11,$472.31,$60.53
Arizona-Phoenix,"$330,344","$1,035 / month",$161.12 / month,$171.75 / month,$2.39 / gallon,$3.24,$1.89,$1.76,$3.27,$4.13,$108.16,$92.92,$101.35,$430.86,$48.23
Colorado-Denver,"$463,245","$1,451 / month",$156.76 / month,$167.10 / month,$2.75 / gallon,$3.26,$1.90,$1.77,$3.29,$4.15,$116.58,$100.15,$109.24,$464.37,$55.28


In [33]:
for x in western_data:
   western_data[x] = western_data[x].str.replace("$","")
   western_data[x] = western_data[x].str.replace("month","")
   western_data[x] = western_data[x].str.replace("/","")
   western_data[x] = western_data[x].str.replace("gallon","")
   western_data[x] = western_data[x].str.replace(",","")
   western_data[x] = pd.to_numeric(western_data[x])
western_data.head()

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
STATES,,,,,,,,,,,,,,,
Washington-Seattle,673874,2111,206.39,220.00,3.47,4.22,2.46,2.29,4.26,5.37,137.50,118.13,128.85,547.72,69.64
Oregon-Portland,620783,1945,130.77,139.40,2.78,3.84,2.23,2.09,3.87,4.88,118.57,101.86,111.11,472.31,60.53
Arizona-Phoenix,330344,1035,161.12,171.75,2.39,3.24,1.89,1.76,3.27,4.13,108.16,92.92,101.35,430.86,48.23
Colorado-Denver,463245,1451,156.76,167.10,2.75,3.26,1.90,1.77,3.29,4.15,116.58,100.15,109.24,464.37,55.28


In [34]:
region = pd.DataFrame(western_data.mean().round(2))
region_t = region.T
region.T #This is tranposing the data

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
0,522061.5,1635.5,163.76,174.56,2.85,3.64,2.12,1.98,3.67,4.63,120.2,103.27,112.64,478.81,58.42


In [35]:
region_n = "Western Region"
the_df = pd.DataFrame(region_t)
the_df['Region'] = region_n

western_data_index = the_df.set_index('Region')
western_data_index

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
Region,,,,,,,,,,,,,,,
Western Region,522061.5,1635.5,163.76,174.56,2.85,3.64,2.12,1.98,3.67,4.63,120.2,103.27,112.64,478.81,58.42


In [36]:
western_data.columns = ['MORTGAGE','RENT','ELECTRICITY','PHONE','GAS','BREAD','MILK','EGGS','BANANAS','POLTRY','PHYSICIAN', 'DENTIST','OPTOMISTRIST', 'RX','VET']
western_data['HLTH & WELL']=western_data['PHYSICIAN'] + western_data['DENTIST'] + western_data['OPTOMISTRIST'] + western_data['RX']
western_data['EAT-IN']=western_data['BREAD'] + western_data['MILK'] + western_data['BANANAS'] + western_data['POLTRY']
western_data = western_data.drop(['BREAD','MILK','EGGS','BANANAS','POLTRY','PHYSICIAN', 'DENTIST', 'OPTOMISTRIST','RX'], axis=1)
western_data

,MORTGAGE,RENT,ELECTRICITY,PHONE,GAS,VET,HLTH & WELL,EAT-IN
STATES,,,,,,,,
Washington-Seattle,673874,2111,206.39,220.00,3.47,69.64,932.20,16.31
Oregon-Portland,620783,1945,130.77,139.40,2.78,60.53,803.85,14.82
Arizona-Phoenix,330344,1035,161.12,171.75,2.39,48.23,733.29,12.53
Colorado-Denver,463245,1451,156.76,167.10,2.75,55.28,790.34,12.60


In [37]:
#Added Region column with "Southern" values in preparation to join with other regions.
region = 'Western'
western_data['REGION'] = [region for _ in range(len(western_data))]
western_data

,MORTGAGE,RENT,ELECTRICITY,PHONE,GAS,VET,HLTH & WELL,EAT-IN,REGION
STATES,,,,,,,,,
Washington-Seattle,673874,2111,206.39,220.00,3.47,69.64,932.20,16.31,Western
Oregon-Portland,620783,1945,130.77,139.40,2.78,60.53,803.85,14.82,Western
Arizona-Phoenix,330344,1035,161.12,171.75,2.39,48.23,733.29,12.53,Western
Colorado-Denver,463245,1451,156.76,167.10,2.75,55.28,790.34,12.60,Western


In [38]:
#Reorderd columns
western_data = western_data[['MORTGAGE', 'RENT', 'ELECTRICITY','GAS', 'EAT-IN','PHONE','HLTH & WELL', 'VET', 'REGION']]
western_data

western_data.to_csv("western.csv")